In [ ]:
!pip install pyspark
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [ ]:
import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

In [ ]:
# Read the CSV file into a DataFrame
df = spark.read.csv('ngram.csv', header=True, inferSchema=True, sep='\t')

# Print the schema of the DataFrame
df.printSchema()
print(df.columns)


root
 |-- ! $17.95: string (nullable = true)
 |-- 1985: integer (nullable = true)
 |-- 12: integer (nullable = true)
 |-- 13: integer (nullable = true)
 |-- 14: integer (nullable = true)

['! $17.95', '1985', '12', '13', '14']


In [ ]:
# prompt: these are the current columns ['! $17.95', '1985', '12', '13', '14']i want t

# Rename the columns to meaningful names
df = df.withColumnRenamed('! $17.95', 'Ngram')
df = df.withColumnRenamed('1985', 'Year')
df = df.withColumnRenamed('12', 'Count')
df = df.withColumnRenamed('13', 'Pages')
df = df.withColumnRenamed('14', 'Books')

# Print the updated schema and columns
df.printSchema()
print(df.columns)


root
 |-- Ngram: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Count: integer (nullable = true)
 |-- Pages: integer (nullable = true)
 |-- Books: integer (nullable = true)

['Ngram', 'Year', 'Count', 'Pages', 'Books']


In [ ]:
df.createOrReplaceTempView("ngram")
spark.sql("SELECT * FROM ngram").show()


+--------+----+-----+-----+-----+
|   Ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
|! $17.95|1993|    2|    2|    2|
|! $17.95|1995|    1|    1|    1|
|! $17.95|1996|    4|    2|    2|
|! $17.95|1997|    6|    5|    5|
|! $17.95|1998|    4|    3|    3|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2001|    5|    4|    4|
|! $17.95|2002|    5|    5|    5|
|! $17.95|2003|    2|    2|    2|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|! $17.95|2006|    5|    5|    5|
|! $17.95|2007|    2|    2|    2|
|! $17.95|2008|    2|    2|    2|
|    ! 09|1780|    1|    1|    1|
+--------+----+-----+-----+-----+
only showing top 20 rows



In [ ]:
# SQL
df.filter(df['Count'] > 5).show()

# Spark API
df.where(df['Count'] > 5).show()

+--------+----+-----+-----+-----+
|   Ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
|    ! 09|2000|   11|    9|    9|
|    ! 09|2004|   14|   14|   14|
|    ! 09|2005|   13|   13|   13|
+--------+----+-----+-----+-----+

+--------+----+-----+-----+-----+
|   Ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|199

In [ ]:
# Spark SQL
spark.sql("SELECT Year, SUM(Count) AS Total FROM ngram GROUP BY Year").show()

# Spark API
df.groupBy("Year").agg({"Count": "sum"}).show()

+----+-----+
|Year|Total|
+----+-----+
|1829|    3|
|1990|    2|
|1903|    1|
|1884|    5|
|1888|    2|
|1924|    1|
|2003|    4|
|1823|    1|
|2007|    4|
|1869|    1|
|1892|    2|
|1889|    2|
|1927|    1|
|1866|    1|
|1877|    2|
|2006|   10|
|1908|    2|
|1925|    2|
|1824|    1|
|1848|    1|
+----+-----+
only showing top 20 rows

+----+----------+
|Year|sum(Count)|
+----+----------+
|1829|         3|
|1990|         2|
|1903|         1|
|1884|         5|
|1888|         2|
|1924|         1|
|2003|         4|
|1823|         1|
|2007|         4|
|1869|         1|
|1892|         2|
|1889|         2|
|1927|         1|
|1866|         1|
|1877|         2|
|2006|        10|
|1908|         2|
|1925|         2|
|1824|         1|
|1848|         1|
+----+----------+
only showing top 20 rows



In [ ]:
# Spark SQL
spark.sql("SELECT Year, ngram, MAX(Count) AS MaxCount FROM ngram GROUP BY Year, ngram").show()

# Spark API
df.groupBy("Year", "ngram").agg({"Count": "max"}).show()

+----+--------+--------+
|Year|   ngram|MaxCount|
+----+--------+--------+
|2002|    ! 09|       5|
|1932|    ! 09|       1|
|1880|    ! 09|       2|
|1935|    ! 09|       2|
|1999|    ! 09|      11|
|1829|    ! 09|       3|
|1999|! $17.95|      11|
|1887|    ! 09|       1|
|2007|    ! 09|       2|
|1899|    ! 09|       6|
|1857|    ! 09|       2|
|2002|! $17.95|       5|
|1929|    ! 09|       1|
|1803|    ! 09|       1|
|1905|    ! 09|       4|
|2000|    ! 09|      11|
|1881|    ! 09|       3|
|1885|    ! 09|       2|
|1831|    ! 09|       1|
|1933|    ! 09|       1|
+----+--------+--------+
only showing top 20 rows

+----+--------+----------+
|Year|   ngram|max(Count)|
+----+--------+----------+
|2002|    ! 09|         5|
|1932|    ! 09|         1|
|1880|    ! 09|         2|
|1935|    ! 09|         2|
|1999|    ! 09|        11|
|1829|    ! 09|         3|
|1999|! $17.95|        11|
|1887|    ! 09|         1|
|2007|    ! 09|         2|
|1899|    ! 09|         6|
|1857|    ! 09|        

In [ ]:
spark.sql("""
SELECT ngram, COUNT(DISTINCT Year) AS num_years
FROM ngram
GROUP BY ngram
HAVING num_years >= 20
""").show()

df.groupBy("Ngram").count().filter("count >= 20").show()


+-----+---------+
|ngram|num_years|
+-----+---------+
| ! 09|      100|
+-----+---------+

+-----+-----+
|Ngram|count|
+-----+-----+
| ! 09|  100|
+-----+-----+



In [ ]:
  # Spark API
df.filter(df['Ngram'].like('!% 09%')).show()

# Spark SQL
spark.sql("SELECT * FROM ngram WHERE Ngram LIKE '!% 09%'").show()


+-----+----+-----+-----+-----+
|Ngram|Year|Count|Pages|Books|
+-----+----+-----+-----+-----+
| ! 09|1780|    1|    1|    1|
| ! 09|1803|    1|    1|    1|
| ! 09|1806|    1|    1|    1|
| ! 09|1823|    1|    1|    1|
| ! 09|1824|    1|    1|    1|
| ! 09|1825|    1|    1|    1|
| ! 09|1829|    3|    3|    3|
| ! 09|1830|    2|    2|    2|
| ! 09|1831|    1|    1|    1|
| ! 09|1833|    1|    1|    1|
| ! 09|1834|    4|    4|    4|
| ! 09|1836|    1|    1|    1|
| ! 09|1839|    1|    1|    1|
| ! 09|1840|    1|    1|    1|
| ! 09|1841|    2|    2|    2|
| ! 09|1845|    1|    1|    1|
| ! 09|1847|    2|    2|    2|
| ! 09|1848|    1|    1|    1|
| ! 09|1856|    1|    1|    1|
| ! 09|1857|    2|    2|    2|
+-----+----+-----+-----+-----+
only showing top 20 rows

+-----+----+-----+-----+-----+
|Ngram|Year|Count|Pages|Books|
+-----+----+-----+-----+-----+
| ! 09|1780|    1|    1|    1|
| ! 09|1803|    1|    1|    1|
| ! 09|1806|    1|    1|    1|
| ! 09|1823|    1|    1|    1|
| ! 09|1824| 

In [ ]:
spark.sql("""
  select Ngram from ngram
  group by Ngram
  having count(distinct(Year)) = (select count(distinct(Year)) from ngram)
""").show()

distinct_year_count = df.select("Year").distinct().count()

df.groupBy("Ngram") \
                    .agg(F.countDistinct("Year").alias("DistinctYearCount")) \
                    .filter(F.col("DistinctYearCount") == distinct_year_count) \
                    .select("Ngram").show()

+-----+
|Ngram|
+-----+
| ! 09|
+-----+

+-----+
|Ngram|
+-----+
| ! 09|
+-----+



In [ ]:
df.groupBy("Year", "Ngram") \
  .agg(F.sum("Pages").alias("TotalPages"), F.sum("Books").alias("TotalBooks")) \
  .orderBy("Ngram") \
  .show()

spark.sql("""
  SELECT Year, Ngram, SUM(Pages) AS TotalPages, SUM(Books) AS TotalBooks
  FROM ngram
  GROUP BY Year, Ngram
  ORDER BY Ngram
""").show()


+----+--------+----------+----------+
|Year|   Ngram|TotalPages|TotalBooks|
+----+--------+----------+----------+
|1987|! $17.95|         1|         1|
|1995|! $17.95|         1|         1|
|2005|! $17.95|        13|        13|
|1990|! $17.95|         1|         1|
|2001|! $17.95|         4|         4|
|2007|! $17.95|         2|         2|
|1996|! $17.95|         2|         2|
|2000|! $17.95|         9|         9|
|1993|! $17.95|         2|         2|
|2002|! $17.95|         5|         5|
|1992|! $17.95|         5|         5|
|2004|! $17.95|        14|        14|
|2003|! $17.95|         2|         2|
|1999|! $17.95|        10|        10|
|2006|! $17.95|         5|         5|
|2008|! $17.95|         2|         2|
|1997|! $17.95|         5|         5|
|1991|! $17.95|         1|         1|
|1998|! $17.95|         3|         3|
|1803|    ! 09|         1|         1|
+----+--------+----------+----------+
only showing top 20 rows

+----+--------+----------+----------+
|Year|   Ngram|TotalPage

In [ ]:
df.groupBy("Year", "Ngram") \
  .agg(F.countDistinct("Ngram").alias("DistinctNgramCount")) \
  .groupBy("Year") \
  .agg(F.sum("DistinctNgramCount").alias("TotalDistinctNgrams")) \
  .sort("Year", ascending=False) \
  .show()


+----+-------------------+
|Year|TotalDistinctNgrams|
+----+-------------------+
|2008|                  2|
|2007|                  2|
|2006|                  2|
|2005|                  2|
|2004|                  2|
|2003|                  2|
|2002|                  2|
|2001|                  2|
|2000|                  2|
|1999|                  2|
|1998|                  2|
|1997|                  2|
|1996|                  2|
|1995|                  2|
|1993|                  2|
|1992|                  2|
|1991|                  2|
|1990|                  2|
|1987|                  2|
|1985|                  1|
+----+-------------------+
only showing top 20 rows



In [ ]:
spark.sql("""
  SELECT Year, SUM(DISTINCT(Ngram)) AS TotalDistinctNgrams
  FROM ngram
  GROUP BY Year
  ORDER BY Year DESC
""").show()


+----+-------------------+
|Year|TotalDistinctNgrams|
+----+-------------------+
|2008|               null|
|2007|               null|
|2006|               null|
|2005|               null|
|2004|               null|
|2003|               null|
|2002|               null|
|2001|               null|
|2000|               null|
|1999|               null|
|1998|               null|
|1997|               null|
|1996|               null|
|1995|               null|
|1993|               null|
|1992|               null|
|1991|               null|
|1990|               null|
|1987|               null|
|1985|               null|
+----+-------------------+
only showing top 20 rows

